In [42]:
import json
import re
from nltk import word_tokenize
from collections import defaultdict, Counter
import nltk
import numpy as np

In [2]:
with open("dump_small.jsonln", 'r') as file:
    data = [json.loads(line) for line in file]

In [226]:
print(limpa_cat(data[0]["body"]))

{{Multitag|mnot|rev|data=outubro de 2019}}
{{Info/Nome
|imagem= Napoli BW 2013-05-16 16-24-01.jpg
|legenda= [[Alexandre, o Grande]], uma das pessoas notórias de nome '''Alexandre'''.
|gênero= Masculino
|significado= Protetor do Homem
|origem= Do [[Língua grega|grego]] Αλέξανδρος
|relativos= Alex; Alexander; Alessandro; Sandro
}}
'''Alexandre''' é um [[prenome]] popular da [[língua portuguesa]]. É [[cognato]] ao nome [[Alexander]], da [[língua inglesa]]. Em [[países lusófonos]], pessoas chamadas Alexandre são normalmente [[Alcunha|apelidadas]] de [[Alex]].

== Origem ==
O nome deriva do [[latim]] ''Alexander'', [[romanização]] do nome [[Língua grega|grego]] Αλέξανδρος (''Aléksandros''). [[Etimologia|Etimologicamente]], o nome seria um composto do [[verbo]] ''ἀλέξειν'' (''aléksein''), "defender", e o [[substantivo]] ''ἀνδρός'' (''andrós''), [[genitivo]] de ''ἀνήρ'' (''anēr''), "homem". Assim, pode ser traduzido como "protetor da Humanidade".{{carece de fontes|data=abril de 2017}}

O term

In [225]:
def limpa_cat(texto):
    pattern = r"""\[\[Categoria:.*?\]\]"""Categoria
    repl = r""
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

def limpa_ref(texto):
    pattern = r"""<ref>.*?</ref>"""
    repl = r""
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

def limpa_aspas(texto):
    pattern = r"""(['"]+)(.*?)\1"""
    repl = r"\2"
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

def limpa_url(texto):
    # Regex obtida de https://www.geeksforgeeks.org/python-check-url-string/
    pattern = r"""
        (?i)  # Ignore case.
        \b  # Inicio de palavra.
        (?:
            https?://
        |
            www
            \d{0,3}
            [.]
        |
            [a-z0-9.\-]+
            [.]
            [a-z]{2,4}
            /
        )
        (?:
            [^\s()<>]+
        |
            \(
            (?:
                [^\s()<>]+
            |
                \(
                [^\s()<>]+
                \)
            )*
            \)
        )+
        (?:
            \(
            (?:
                [^\s()<>]+
            |
                \(
                [^\s()<>]+
                \)
            )*
            \)
        |
            [^\s`!()\[\]{};:'\".,<>?«»“”‘’]
        )
    """
    repl = ''
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

def limpa_templates(texto):
    conta = 0
    spans_proibidos = []
    for item in re.finditer(r'{{|}}', texto):
        if item[0] == '{{':
            if conta == 0:
                inicio = item.span()[0]
            conta += 1
        else:
            conta -= 1
            if conta == 0:
                fim = item.span()[1]
                spans_proibidos.append((inicio, fim))
    texto_limpo = ''
    inicio = 0
    for span in spans_proibidos:
        fim, novo_inicio = span
        texto_limpo += texto[inicio:fim]
        inicio = novo_inicio
    texto_limpo += texto[inicio:]
    return texto_limpo

def limpa_wikilinks(texto):
    pattern = r'''
        \[\[
        (?:
            [^|]*?\|
        )*?
        (
            [^|]*?
        )
        \]\]
    '''
    repl = r'\1'
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

def minusculas(tokens):
    return [token.lower() for token in tokens]

def remove_digitos(tokens):
    matcher = re.compile('[^\d]*')
    return [token for token in tokens if matcher.fullmatch(token)]

def pega_palavras(tokens):
    matcher = re.compile('[a-záéíóúçâêôãõà]+(?:-[a-záéíóúçâêôãõà]+)*')
    return [token for token in tokens if matcher.fullmatch(token)]

def remove_stopwords(tokens):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    return [token for token in tokens if token not in stopwords]
    
def limpa_texto(texto):
    texto = limpa_aspas(texto)
    texto = limpa_ref(texto)
    texto = limpa_url(texto)
    texto = limpa_templates(texto)
    texto = limpa_wikilinks(texto)
    texto = limpa_cat(texto)
    return texto

def limpa_tokens(tokens):
    tokens = minusculas(tokens)
    tokens = remove_digitos(tokens)
    tokens = pega_palavras(tokens)
    return tokens

In [222]:
all_words = []
# N = 0
# all_words_per_doc_single = []

for item in data:
    texto = item['body']
    texto = limpa_texto(texto)
    tokens = word_tokenize(texto)
    tokens = limpa_tokens(tokens)
    tokens = remove_stopwords(tokens)
    all_words += tokens
#     N += len(tokens)
#     all_words_per_doc_single += set(tokens)


all_words_clean = limpa_tokens(all_words)
# doc_counts = Counter(all_words_per_doc_single)

print(len(all_words_clean))

<ipython-input-221-331202656549>:4: FutureWarning: Possible nested set at position 1
  matcher = re.compile(pattern, re.VERBOSE)


KeyboardInterrupt: 

In [214]:
word_counts = defaultdict(int)
for w in all_words_clean:
    word_counts[w] += 1

word_counts = Counter(all_words_clean)
word_counts_list = list(word_counts.items())
word_counts_list_sorted = sorted(word_counts_list, key=lambda x: (-x[1], x[0]))[:3000]

In [215]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
UPPERCASE = [chr(x) for x in range(ord('A'), ord('Z') + 1)]
LOWERCASE_OTHERS = ['ç', 'á', 'â', ]  # etc.
UPPERCASE_OTHERS = [x.upper() for x in LOWERCASE_OTHERS]
LETTERS = LOWERCASE + UPPERCASE + LOWERCASE_OTHERS + UPPERCASE_OTHERS

def edit1(text):
    words = []
    
    # Fase 1: as remoçoes.
    for p in range(len(text)):
        new_word = text[:p] + text[p + 1:]
        if len(new_word) > 0:
            words.append(new_word)
        
    # Fase 2: as adições.
    for p in range(len(text) + 1):
        for c in LETTERS:
            new_word = text[:p] + c + text[p:]
            words.append(new_word)
    
    # Fase 3: as substituições.
    for p in range(len(text)):
        orig_c = text[p]
        for c in LETTERS:
            if orig_c != c:
                new_word = text[:p] + c + text[p + 1:]
                words.append(new_word)
    
    return set(words)

def edit2(text):
    words1 = edit1(text)
    words2 = set()
    for w in words1:
        candidate_words2 = edit1(w)
        candidate_words2 -= words1
        words2.update(candidate_words2)
    words2 -= set([text])
    return words2


In [216]:
vocab = word_counts_list_sorted[:10000]
vocab = dict(vocab)

In [217]:
word = 'shotmas'

In [218]:
if word in vocab:
    candidatos = [word]
else:
    candidatos = []
candidatos += \
    [w for w in edit1(word) if w in vocab] \
    + [w for w in edit2(word) if w in vocab] \
    + [word]

In [219]:
candidatos

['thomas', 'shotmas']

In [220]:
vocab

{'categoria': 28122,
 'ref': 20273,
 'br': 11351,
 'center': 11264,
 'the': 8873,
 'of': 5639,
 'ser': 5636,
 'small': 5524,
 'anos': 4952,
 'nbsp': 4382,
 'cidade': 4337,
 'brasil': 4303,
 'left': 4283,
 'sobre': 4144,
 'estado': 4134,
 'rio': 4021,
 'ano': 3774,
 'estados': 3499,
 'onde': 3491,
 'grande': 3484,
 'durante': 3288,
 'primeira': 3253,
 'janeiro': 3243,
 'após': 3225,
 'primeiro': 3109,
 'ligações': 3105,
 'externas': 3064,
 'dois': 3051,
 'parte': 3012,
 'história': 2971,
 'and': 2949,
 'unidos': 2949,
 'mount': 2937,
 'paulo': 2925,
 'lemmon': 2910,
 'sendo': 2788,
 'in': 2748,
 'segundo': 2728,
 'nome': 2709,
 'dia': 2694,
 'peak': 2694,
 'kitt': 2685,
 'spacewatch': 2679,
 'filmes': 2655,
 'nova': 2630,
 'jogos': 2627,
 'outros': 2608,
 'região': 2607,
 'família': 2567,
 'ainda': 2523,
 'grupo': 2517,
 'sul': 2491,
 'futebol': 2486,
 'série': 2440,
 'final': 2370,
 'tempo': 2331,
 'área': 2313,
 'três': 2297,
 'portugal': 2223,
 'nacional': 2200,
 'contra': 2175,
 'fi

In [200]:
def P(word, vocab): 
    try:
        return vocab[word] / sum(vocab.values())
    except KeyError:
        return 0
        
def candidates(word, vocab):
    if word in vocab:
        candidatos = [word]
    else:
        candidatos = []
    candidatos += \
        [w for w in edit1(word) if w in vocab] \
        + [w for w in edit2(word) if w in vocab] \
        + [word]
    return candidatos

def correction(word, vocab):
    max_p = max(candidates(word, vocab), key=lambda w: P(w,vocab))
    return max_p

def correct_text(text, vocab):
    text_final = ""
    stopwords = frozenset(nltk.corpus.stopwords.words('portuguese'))
    for p in text.split(" "):
        if p in stopwords:
            text_final += p + " "
        else:
            text_final += correction(p, vocab) + " "
    return text_final[:-1]

In [203]:
# text = str(input("Texto: "))
text = "shotmas"

In [204]:
correct_text(text, vocab)

'thomas'